In [8]:
import torch
from torchtext import data
from torchtext.data import BucketIterator
import torch.optim as optim
from transformers import AutoTokenizer

import pandas as pd
import numpy as np
import os

from dotenv import load_dotenv, find_dotenv
import warnings
warnings.filterwarnings("ignore")

In [9]:
load_dotenv(find_dotenv())

True

In [10]:
project_root = os.environ.get("PROJECT_ROOT")
data_dir = os.environ.get("DATA_DIR")
dataset_fullpath = os.path.join(project_root, data_dir, "output")
small_dataset_output = os.path.join(dataset_fullpath, "model_output", "small_patches")
large_dataset_output = os.path.join(dataset_fullpath, "model_output", "large_patches")

In [11]:
large_patches_df = pd.read_csv(os.path.join(dataset_fullpath, "large-patches.csv"))
small_patches_df = pd.read_csv(os.path.join(dataset_fullpath, "small-patches.csv")) 

# Processing small patches dataset

In [12]:
torch.manual_seed(42)

In [13]:

def csv_train_test_split(save_path, df):

    df = df.sample(frac=1).reset_index(drop=True)  

    df['numerical_label'] = df['label'].replace({'correct': 1, 'overfitting': -1})

    total_rows = len(df)
    first = int(0.8 * total_rows)
    second = int(0.9 * total_rows)

    train_df = df.iloc[:first]
    val_df = df.iloc[first:second]
    test_df = df.iloc[second:]

    train_df.to_csv(os.path.join(save_path, "train.csv"), index=False)
    val_df.to_csv(os.path.join(save_path, "val.csv"), index=False)
    test_df.to_csv(os.path.join(save_path, "test.csv"), index=False)

    return train_df, val_df, test_df

In [14]:
train_df, val_df, test_df = csv_train_test_split(small_dataset_output, small_patches_df)

Using codebert pretrained tokenizer

In [15]:
# tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
pad_index = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
unk_index = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

## Loading the Dataset and Building the Fields

In [16]:
MAX_SEQ_LEN = 512

In [17]:
# buggy # patch # label
BUGGY_CODE = data.Field(use_vocab =False, tokenize = tokenizer.encode, pad_token=pad_index, unk_token=unk_index, fix_length=MAX_SEQ_LEN)
PATCH_CODE = data.Field(use_vocab =False, tokenize= tokenizer.encode, pad_token=pad_index, unk_token=unk_index, fix_length=MAX_SEQ_LEN)
LABEL = data.Field(use_vocab=False, sequential=False)

In [18]:
fields = [("dataset", None),
        ("tool", None),
        ("buggy", BUGGY_CODE),
        ("patch", PATCH_CODE),
        ("label", None),
        ("numerical_label", LABEL)]

In [19]:
fields

[('dataset', None),
 ('tool', None),
 ('buggy', <torchtext.data.field.Field at 0x7fbb2ea6e9e0>),
 ('patch', <torchtext.data.field.Field at 0x7fbb2ea6f190>),
 ('label', None),
 ('numerical_label', <torchtext.data.field.Field at 0x7fbb2ea6efe0>)]

In [21]:
train = data.TabularDataset(
        path= os.path.join(small_dataset_output, "train.csv"),
        format="csv",
        skip_header=True,
        fields=fields
        )
val =  data.TabularDataset(
        path= os.path.join(small_dataset_output, "val.csv"),
        format="csv",
        skip_header=True,
        fields=fields
        )
test = data.TabularDataset(
        path= os.path.join(small_dataset_output, "test.csv"),
        format="csv",
        skip_header=True,
        fields=fields
        )

In [22]:

print(f"Number of training examples: {len(train.examples)}")
print(f"Number of validation examples: {len(val.examples)}")
print(f"Number of testing examples: {len(test.examples)}")

Number of training examples: 946
Number of validation examples: 118
Number of testing examples: 119


In [23]:
print(vars(train.examples[5]))

{'buggy': [0, 15110, 5997, 45200, 40462, 48176, 49162, 28874, 13984, 1640, 46674, 26907, 6, 6979, 22523, 6, 16224, 11212, 42379, 43, 25522, 114, 36, 36097, 8061, 321, 43, 25522, 1306, 15791, 18583, 1640, 10799, 2055, 22523, 4397, 26602, 7031, 5457, 36, 46134, 45994, 23796, 17487, 120, 49302, 39645, 43048, 4832, 26907, 4, 560, 34222, 49291, 6979, 7031, 45280, 5457, 7031, 4, 16096, 47006, 114, 36, 6031, 45280, 49095, 22523, 43, 25522, 1437, 2], 'patch': [0, 1290, 2407, 15791, 18583, 48461, 10799, 43, 2055, 204, 4397, 1437, 2], 'numerical_label': '-1'}


## Building the data for the model

In [24]:
device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [29]:
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x : len(x.__dict__),
    sort_within_batch = False,
    device = device)


In [32]:
for i, batch in enumerate(train_iterator):
    print("i:",i)
    one_bug_sample = batch.buggy
    one_patch_sample = batch.patch
    label = batch.numerical_label
    print(f"buggy: {batch.buggy}")
    print(f"buggy size: {batch.buggy.T.size()}")
    print(f"patch: {batch.patch}")
    print(f"patch size: {batch.patch.T.size()}")
    print(f"label: {batch.numerical_label.size()}")
    break

i: 0
buggy: tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [46674,  1594,  1594,  ...,  3983,  1594, 15110],
        [40118,    36,    36,  ..., 48547,    36, 26602],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:2')
buggy size: torch.Size([32, 512])
patch: tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [46674,  1594,  1594,  ...,  1594,  1594, 15117],
        [40118,    36, 48461,  ..., 41006, 48614, 29702],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:2')
patch size: torch.Size([32, 512])
label: torch.Size([32])


In [33]:
for i, batch in enumerate(valid_iterator):
    print("i:",i)
    one_bug_sample = batch.buggy
    one_patch_sample = batch.patch
    label = batch.numerical_label
    print(f"buggy: {batch.buggy}")
    print(f"buggy size: {batch.buggy.T.size()}")
    print(f"patch: {batch.patch}")
    print(f"patch size: {batch.patch.T.size()}")
    print(f"label: {batch.numerical_label.size()}")
    break

i: 0
buggy: tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [24303,  1594,  2544,  ..., 10339, 24303, 15110],
        [  403,    36, 13561,  ...,   131,   940,   230],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:2')
buggy size: torch.Size([32, 512])
patch: tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [ 1594,   180, 20235,  ...,  1594,     2,  1437],
        [48461, 10643,    36,  ..., 48461,     1,     2],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:2')
patch size: torch.Size([32, 512])
label: torch.Size([32])


In [181]:
tokenizer.convert_ids_to_tokens(one_bug_sample[:, 1])

['<s>',
 'return',
 'Ġread',
 'Object',
 ';',
 'Ġ}',
 'Ġcatch',
 'Ġ(',
 'Class',
 'Not',
 'Found',
 'Exception',
 'Ġex',
 ')',
 'Ġ{',
 'Ġthrow',
 'Ġnew',
 'ĠSerial',
 'ization',
 'Exception',
 '("',
 'Class',
 'Not',
 'Found',
 'Exception',
 'Ġwhile',
 'Ġreading',
 'Ġcl',
 'oned',
 'Ġobject',
 'Ġdata',
 '",',
 'Ġex',
 ');',
 'Ġ}',
 'Ġcatch',
 'Ġ(',
 'IO',
 'Exception',
 'Ġex',
 ')',
 'Ġ{',
 'Ġthrow',
 'Ġnew',
 'ĠSerial',
 'ization',
 'Exception',
 '("',
 'IO',
 'Exception',
 'Ġwhile',
 'Ġreading',
 'Ġcl',
 'oned',
 'Ġobject',
 'Ġdata',
 '",',
 'Ġex',
 ');',
 'Ġ}',
 'Ġfinally',
 'Ġ{',
 'Ġ',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<p

In [34]:
vocab_size = len(tokenizer)

print("Vocabulary Size:", vocab_size) 

Vocabulary Size: 50265


## Building the model architecture

In [37]:
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [38]:
import torch.nn as nn
import math

class CodeEncoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, num_heads, num_layers, dropout=0.1):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.pos_encoder = PositionalEncoding(d_model=embedding_dim)
    self.embedding_dim = embedding_dim

    self.transformer_encoder = nn.TransformerEncoder(
        nn.TransformerEncoderLayer(
            d_model=embedding_dim, nhead=num_heads, dim_feedforward=4 * embedding_dim, dropout=dropout
        ),
        num_layers=num_layers
    )

  def forward(self, code_input):
    embedded_code = self.embedding(code_input) * math.sqrt(self.embedding_dim) 
    encoded_code = self.pos_encoder(embedded_code)
    output = self.transformer_encoder(encoded_code)  
  
    return output

In [54]:
class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf

        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda:2')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        # modified to handle edge cases when there is no positive pair
        # for an anchor point. 
        # Edge case e.g.:- 
        # features of shape: [4,1,...]
        # labels:            [0,1,1,2]
        # loss before mean:  [nan, ..., ..., nan] 
        mask_pos_pairs = mask.sum(1)
        mask_pos_pairs = torch.where(mask_pos_pairs < 1e-6, 1, mask_pos_pairs)
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask_pos_pairs

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [55]:
def init_train_var(model):
    criterion = SupConLoss()
    optimizer =  optim.SGD(model.parameters(), lr= 0.01)
    return criterion, optimizer

In [56]:
def train(model, train_iterator, valid_iterator, batch_size):
    nb_epochs = 20
    criterion, optimizer = init_train_var(model)

    train_steps = len(train_iterator.dataset) // batch_size
    val_steps = len(valid_iterator.dataset) // batch_size

    model.train()
    for epoch in range(nb_epochs):
        epoch_train_loss = 0
        epoch_val_loss = 0

        # train_correct = 0  not using it now since there is no patch classifier
        # val_correct = 0

        for batch in train_iterator:
            buggy_tensor, patch_tensor, labels = batch.buggy.T, batch.patch.T, batch.numerical_label # data already in device

            optimizer.zero_grad()
            buggy_embd = model(buggy_tensor)
            patch_embd = model(patch_tensor)

            feature = torch.norm(buggy_embd - patch_embd, dim=2, keepdim=True)
    
            loss = criterion(feature, labels) # feature, label
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss

        print(epoch_train_loss)
        model.eval()
        with torch.no_grad():
            for batch in valid_iterator:
                buggy_tensor, patch_tensor, labels = batch.buggy.T, batch.patch.T, batch.numerical_label
                buggy_embd = model(buggy_tensor)
                patch_embd = model(patch_tensor)

                feature = torch.norm(buggy_embd - patch_embd, dim=2, keepdim=True)
                loss = criterion(feature, labels)

                epoch_val_loss += loss

        mean_train_loss = epoch_train_loss / train_steps
        mean_val_loss = epoch_val_loss / val_steps

        print("[INFO] EPOCH: {}/{}".format(epoch + 1, nb_epochs))
        print("Train loss: {:.6f}".format(mean_train_loss))
        print("Val loss: {:.6f}\n".format(mean_val_loss))

In [57]:
code_encoder = CodeEncoder(vocab_size, embedding_dim=300, num_heads=4, num_layers=4)
code_encoder.to(device)
train(code_encoder, train_iterator, valid_iterator, BATCH_SIZE)

tensor(4884.5879, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 1/20
Train loss: 168.434067
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 2/20
Train loss: 10.018778
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 3/20
Train loss: 10.018778
Val loss: 12.813759

tensor(290.5445, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 4/20
Train loss: 10.018777
Val loss: 12.813759

tensor(290.5445, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 5/20
Train loss: 10.018777
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 6/20
Train loss: 10.018778
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 7/20
Train loss: 10.018778
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad_fn=<AddBackward0>)
[INFO] EPOCH: 8/20
Train loss: 10.018778
Val loss: 12.813759

tensor(290.5446, device='cuda:2', grad

In [73]:
total_params = sum(p.numel() for p in code_encoder.parameters())
total_params

19415100